In [1]:
from rlbirdv2 import *
from ple import PLE
from IPython.display import Image
import imageio
import mdptoolbox
from keras.models import Sequential, load_model
from keras.layers import Conv2D, Flatten, Dense, Activation
from keras.optimizers import Adam
from keras import backend as kerasBackend
from ple import PLE
import pickle

x = [i for i in range(3)]
y = [i for i in range(3)]
y.reverse()

# List of coordinates for islands
island = [(0, 0)]
birdStart = (0, 0)

TILESIZE = 40
SCREEN_WIDTH = TILESIZE*(len(x)+3)
SCREEN_HEIGHT = TILESIZE * len(y)

reward = {'win' : 1,
         'lose' : 0}

# import specific Game -------------------------
file = open("../RLv1/fish2",'rb')
fish = pickle.load(file)
# ----------------------------------------------

# run.py
pygame.init()
game = RLBird(width = SCREEN_WIDTH, height = SCREEN_HEIGHT, x = x, y = y,\
              init_bird_position = birdStart, island_position = island, \
              energyMax = 10, catchMax = 2, costMove = -1, costDive = -1, factorFishFly = 0.25,\
              reward = reward)
game.init()
game.updateFishMap(fish.copy())

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
couldn't import doomish
Couldn't import doom


Using TensorFlow backend.


### Human Policy 

In [2]:
pygame.init()
p = PLE(game, fps=100, frame_skip=1, num_steps=1, force_fps=True, display_screen=True, reward_values = reward)

p.init()
p.reset_game()
game.updateFishMap(fish.copy())

dt = 0
while(not p.game_over()):
        game.step(dt)
        pygame.display.update()
        dt += 1
print(game.getScore())
pygame.quit()

SystemExit: 

/home/amdroy/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
fish

array([[0.        , 0.01552608, 0.30287186],
       [0.1562188 , 0.25212845, 0.3765251 ],
       [0.26855578, 0.52494901, 0.43451012]])

### Deep Q Learning

In [7]:
def epsilon(step):
    """
    Epsilon for exploration/exploitation trade-off
    """
    if step < INITIAL_EXPLORATION:
        return 1
    elif step < EXPLORATION_STEPS:
        return INITIAL_EPSILON + (FINAL_EPSILON - INITIAL_EPSILON)/(EXPLORATION_STEPS-INITIAL_EXPLORATION) * (step-INITIAL_EXPLORATION)
    else:
        return FINAL_EPSILON
    
def createDQN(game):
    """
    Create deep Q network
    """
    # Neural network
    dqn = Sequential()
    dqn.add(Dense(units = 10, input_dim = game.listBirdStates.all.shape[1] , activation='relu'))
    dqn.add(Dense(units = 10, activation='relu'))
    dqn.add(Dense(units = len(game.listAction) , activation='relu'))
    dqn.add(Activation('linear'))
    
    dqn.compile(optimizer=Adam(lr=LEARNING_RATE), loss='mean_squared_error')
    return dqn


def epsilonGreedy(dqn, game, x, step):
    """
    Epsilon-greedy action
    """  
    if np.random.rand() < epsilon(step):
        a = np.random.choice(len(game.listAction))
    else:
        a = np.argmax(dqn.predict(np.array([x])))
    return a


In [8]:
class MemoryBuffer:
    """
    An experience replay buffer using numpy arrays
    """
    def __init__(self, length):
        self.size = 0
        self.length = length
        self.index = 0 
        
        self.states = np.zeros((length,1))
        self.actions = np.zeros((length,1))
        self.rewards = np.zeros((length,1))
        self.states_new = np.zeros((length,1))
        self.ds = np.zeros((length,1))
    
    def append(self, state, a, reward, state_new, d):
        
        self.states[self.index] = state
        self.actions[self.index] = a
        self.rewards[self.index] = reward
        self.states_new[self.index] = state_new
        self.ds[self.index] = d
        
        self.index = (self.index+1) % self.length
        self.size = np.min([self.size+1,self.length])

    def minibatch(self, size):
        indices = np.random.choice(self.size, size=size, replace=False)

        return self.states[indices], self.actions[indices], self.rewards[indices], self.states_new[indices], self.ds[indices]


In [ ]:
import timeit

COMPUTE_NEW_POLICY = True

# Epsilon
INITIAL_EXPLORATION = 1000
EXPLORATION_STEPS   = 1000000
INITIAL_EPSILON     = 1
FINAL_EPSILON       = 1e-2

# Constants
GAMMA        = 0.99
ALPHA        = 0.01
NUMBER_GAMES = 1000000
INTERMEDIATE_SCORE = 1000

# Epsilon
LEARNING_RATE = 0.1

# Weigh Transfer
STEPS_BETWEEN_TRANSFER = 2500
REPLAY_MEM_SIZE = 30000
MINIBATCH_SIZE = 32

# Try to load DQN, or create a new one
if COMPUTE_NEW_POLICY:
    dqnExploration = createDQN(game)
    dqnTarget = dqnExploration
    count = np.zeros((game.listBirdStates.size, len(game.listAction))) # to track update frequencies
else :
    dqnExploration = load_model('dqn.h5')
    dqnTarget = dqnExploration
    count = np.zeros((game.listBirdStates.size, len(game.listAction))) # to track update frequencies
    

score = np.zeros(INTERMEDIATE_SCORE)
replayMemory = MemoryBuffer(REPLAY_MEM_SIZE)
time_start = timeit.default_timer()

kk = 0
for k in range(NUMBER_GAMES):
    if((k+1)%INTERMEDIATE_SCORE==0):
        print('Mean Score after ', k+1, ' Games : ', np.mean(score))
        kk = 0 
        
    p.init()
    p.reset_game()
    game.updateFishMap(fish.copy())
    reward = 0.0
    
    while(not p.game_over()):
        s = game.getBirdState()
        x = game.listBirdStates.state2idx(np.array(s))    
        a = epsilonGreedy(dqnExploration, game, s, k)
        reward = p.act(game.dictAction[game.listAction[a]])
        s_new = game.getBirdState()
        x_new = game.listBirdStates.state2idx(np.array(s_new))  
        
        replayMemory.append(x, a, reward, x_new, p.game_over())
        
        # Train (minibatch)
        if k > INITIAL_EXPLORATION and k > MINIBATCH_SIZE:
            state, action, reward, state_new, d = replayMemory.minibatch(MINIBATCH_SIZE)
            QY = dqnTarget.predict(game.listBirdStates.idx2state(state_new.astype(int)).reshape(MINIBATCH_SIZE,4))
            QYmax = QY.max(1).reshape((MINIBATCH_SIZE, 1))
            update = reward + GAMMA * (1-d) * QYmax
            QX = dqnExploration.predict(game.listBirdStates.idx2state(state.astype(int)).reshape(MINIBATCH_SIZE,4))
            QX[np.arange(MINIBATCH_SIZE), action.astype(int).ravel()] = update.ravel()
            dqnExploration.train_on_batch(x = game.listBirdStates.idx2state(state.astype(int)).reshape(MINIBATCH_SIZE,4), y=QX)

        # Update target dqn
        if k % STEPS_BETWEEN_TRANSFER == 0:
            dqnTarget = dqnExploration
#             dqnExploration.save('dqn.h5')
#             dqnTarget = load_model('dqn.h5')       
        
    score[kk] = game.getScore()
    kk += 1
pygame.quit()

dqn.save('dqn.h5')
time_end = timeit.default_timer()

Mean Score after  1000  Games :  0.026
Mean Score after  2000  Games :  0.021
Mean Score after  3000  Games :  0.023
Mean Score after  4000  Games :  0.029
Mean Score after  5000  Games :  0.026
Mean Score after  6000  Games :  0.039
Mean Score after  7000  Games :  0.022
Mean Score after  8000  Games :  0.017
Mean Score after  9000  Games :  0.019
Mean Score after  10000  Games :  0.022
Mean Score after  11000  Games :  0.039
Mean Score after  12000  Games :  0.032
Mean Score after  13000  Games :  0.019
Mean Score after  14000  Games :  0.038
Mean Score after  15000  Games :  0.032
Mean Score after  16000  Games :  0.027
Mean Score after  17000  Games :  0.025
Mean Score after  18000  Games :  0.032
Mean Score after  19000  Games :  0.019
Mean Score after  20000  Games :  0.029
Mean Score after  21000  Games :  0.023
Mean Score after  22000  Games :  0.023
Mean Score after  23000  Games :  0.022
Mean Score after  24000  Games :  0.028


In [ ]:
update

In [ ]:
## View current policy
p.init()
p.reset_game()
game.updateFishMap(saveFishMap.copy())

images = []
dt = 0
while(not p.game_over()):
    state = game.getGameState()
    idx = game.listStates.state2idx(state)
    Q  = dqn.predict(np.array([state]))
    a = np.argmax(Q)
    reward = p.act(game.dictAction[game.listAction[a]])
    
    pic = './results/screenshot'+str(dt)+'.jpeg'
    pygame.image.save(game.screen, pic)
    images.append(imageio.imread(pic))

    dt +=1
    
imageio.mimsave('./results/DeepRL.gif', images)
pygame.quit()

In [ ]:
with open('./results/DeepRL.gif','rb') as file:
    display(Image(file.read()))